In [1]:
#!pip install textattack[tensorflow,optional]
#!pip install -U datasets

In [2]:
import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC


D:\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._p

In [3]:
from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding, WordSwapWordNet 

from textattack.transformations.composite_transformation import CompositeTransformation

from textattack.attack_recipes.attack_recipe import AttackRecipe

class TFLepsilon(AttackRecipe):

    @staticmethod
    def build(model_wrapper):
        #
        # Swap words with their 50 closest embedding nearest-neighbors.
        # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
        
        ## transformation = CompositeTransformation([WordSwapWordNet(), WordSwapEmbedding(max_candidates=50)])
        transformation = WordSwapEmbedding(max_candidates=50)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
        stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", 
             "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", 
             "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren",
             "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", 
             "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", 
             "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", 
             "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", 
             "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly",
             "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", 
             "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", 
             "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", 
             "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", 
             "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", 
             "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no",
             "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on",
             "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", 
             "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", 
             "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", 
             "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", 
             "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those",
             "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless",
             "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't",
             "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", 
             "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", 
             "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn",
             "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
             "yourselves"]
        )
        # fmt: on
        constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        #
        constraints.append(WordEmbeddingDistance(min_cos_sim=0.7))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
        use_constraint = UniversalSentenceEncoder(
            threshold = 0.840845057,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)


In [4]:
def runAttack(model_wrapper, n, path, target_ds):
    # used TFLepsilon
    attack = TFLepsilon.build(model_wrapper)
    attack_args = textattack.AttackArgs(num_examples=n, log_to_csv=path, shuffle=True, #parallel=True,
                                        checkpoint_interval=5, checkpoint_dir='checkpoints', 
                                        disable_stdout=True)
    attacker = textattack.Attacker(attack, target_ds, attack_args)
    attacker.attack_dataset()

In [5]:
# classifiers
def adaptation(train_set, test_set):
    lr = LogisticRegression(solver='saga', multi_class='multinomial')
    rf = RandomForestClassifier(max_depth=7)
    knn = KNeighborsClassifier(n_neighbors=7)
    svc = SVC(kernel='linear', probability=True)
    
    trainX = train_set.iloc[:,:-1]
    trainY = train_set.iloc[:,-1]
    testX = test_set.iloc[:,:-1]
    testY = test_set.iloc[:,-1]
    
    for key, clf in {'Log Reg':lr, 'RF':rf, 'KNN':knn, 'SVC':svc}.items():
        clf.fit(trainX, trainY) # fit train data
        pred = clf.predict(testX) # predict test set

        # print accuracy
        print(key, (pred==np.array(testY)).sum()/len(pred))

    return lr, rf, knn, svc

In [6]:
target_ds_mr = textattack.datasets.HuggingFaceDataset('rotten_tomatoes', split='test')

textattack: Loading datasets dataset rotten_tomatoes, split test.


In [7]:
suffix = 'sim07.csv'
n_samples = 500

### BERT-based-uncased

In [8]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [9]:
bertmrtrain = pd.read_csv('dataset_embeddings/bertmrtrain.txt', header=None)
bertmrtest = pd.read_csv('dataset_embeddings/bertmrtest.txt', header=None)
lr_bert_mr, rf_bert_mr, knn_bert_mr, svc_bert_mr = adaptation(bertmrtrain, bertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.8105065666041276
RF 0.7326454033771107
KNN 0.6801125703564728
SVC 0.8030018761726079


In [10]:
# custom model wrapper for CLIP
class BERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [11]:
bert_model_wrapper_lr_mr = BERTModelWrapper(bert_tokenizer, bert_model, lr_bert_mr)
bert_model_wrapper_rf_mr = BERTModelWrapper(bert_tokenizer, bert_model, rf_bert_mr)
bert_model_wrapper_knn_mr = BERTModelWrapper(bert_tokenizer, bert_model, knn_bert_mr)
bert_model_wrapper_svc_mr = BERTModelWrapper(bert_tokenizer, bert_model, svc_bert_mr)

In [12]:
runAttack(bert_model_wrapper_lr_mr, n_samples, 'attack_results_new/bertmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacklrsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   1%|▎                         | 5/500 [01:12<2:00:01, 14.55s/it]textattack: Saving checkpoint under "checkpoints\1720197091653.ta.chkpt" at 2024-07-06 00:31:31 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   2%|▍                       | 10/500 [01:19<1:04:49,  7.94s/it]textattack: Saving checkpoint under "checkpoints\1720197098289.ta.chkpt" at 2024-07-06 00:31:38 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   3%|▊                        | 15/500 [01:40<54:07,  6.70s/it]textattack: Saving checkpoint under "checkpoints\1720197119354.ta.chkpt" at 2024-07-06 00:31:59 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 346 / 40 / 99 / 485:  97%|███████████████████▍| 485/500 [14:41<00:27,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1720197900127.ta.chkpt" at 2024-07-06 00:45:00 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 350 / 40 / 100 / 490:  98%|██████████████████▌| 490/500 [14:50<00:18,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1720197909137.ta.chkpt" at 2024-07-06 00:45:09 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 355 / 40 / 100 / 495:  99%|██████████████████▊| 495/500 [14:57<00:09,  1.81s/it]textattack: Saving checkpoint under "checkpoints\1720197916110.ta.chkpt" at 2024-07-06 00:45:16 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 359 / 41 / 100 / 500: 100%|███████████████████| 500/500 [15:09<00:00,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1720197928622.ta.chkpt" at 2024-07-06 00:45:28 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 359 / 41 / 100 / 500: 100%|███████████████████| 500/500 [15:09<00:00,  1.82s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 359    |
| Number of failed attacks:     | 41     |
| Number of skipped attacks:    | 100    |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 8.2%   |
| Attack success rate:          | 89.75% |
| Average perturbed word %:     | 19.08% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 46.98  |
+-------------------------------+--------+


In [13]:
runAttack(bert_model_wrapper_rf_mr, n_samples, 'attack_results_new/bertmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackrfsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 3 / 0 / 2 / 5:   1%|▎                           | 5/500 [00:19<31:25,  3.81s/it]textattack: Saving checkpoint under "checkpoints\1720197948084.ta.chkpt" at 2024-07-06 00:45:48 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 0 / 5 / 10:   2%|▌                         | 10/500 [00:24<20:08,  2.47s/it]textattack: Saving checkpoint under "checkpoints\1720197953696.ta.chkpt" at 2024-07-06 00:45:53 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 0 / 6 / 15:   3%|▊                         | 15/500 [00:37<20:23,  2.52s/it]textattack: Saving checkpoint under "checkpoints\1720197966890.ta.chkpt" at 2024-07-06 00:46:06 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 306 / 43 / 136 / 485:  97%|██████████████████▍| 485/500 [12:45<00:23,  1.58s/it]textattack: Saving checkpoint under "checkpoints\1720198694584.ta.chkpt" at 2024-07-06 00:58:14 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 310 / 43 / 137 / 490:  98%|██████████████████▌| 490/500 [12:56<00:15,  1.58s/it]textattack: Saving checkpoint under "checkpoints\1720198705213.ta.chkpt" at 2024-07-06 00:58:25 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 310 / 43 / 138 / 491:  98%|██████████████████▋| 491/500 [12:56<00:14,  1.58s/it]

[Succeeded / Failed / Skipped / Total] 313 / 43 / 139 / 495:  99%|██████████████████▊| 495/500 [13:05<00:07,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1720198714539.ta.chkpt" at 2024-07-06 00:58:34 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 317 / 43 / 140 / 500: 100%|███████████████████| 500/500 [13:13<00:00,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1720198722665.ta.chkpt" at 2024-07-06 00:58:42 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 317 / 43 / 140 / 500: 100%|███████████████████| 500/500 [13:13<00:00,  1.59s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 317    |
| Number of failed attacks:     | 43     |
| Number of skipped attacks:    | 140    |
| Original accuracy:            | 72.0%  |
| Accuracy under attack:        | 8.6%   |
| Attack success rate:          | 88.06% |
| Average perturbed word %:     | 15.97% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 46.57  |
+-------------------------------+--------+


In [14]:
runAttack(bert_model_wrapper_knn_mr, n_samples, 'attack_results_new/bertmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattackknnsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 3 / 0 / 2 / 5:   1%|▎                           | 5/500 [00:12<19:50,  2.41s/it]textattack: Saving checkpoint under "checkpoints\1720198734820.ta.chkpt" at 2024-07-06 00:58:54 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   2%|▌                         | 10/500 [00:15<13:00,  1.59s/it]textattack: Saving checkpoint under "checkpoints\1720198738739.ta.chkpt" at 2024-07-06 00:58:58 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 1 / 6 / 15:   3%|▊                         | 15/500 [00:37<20:08,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1720198760168.ta.chkpt" at 2024-07-06 00:59:20 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 322 / 16 / 147 / 485:  97%|██████████████████▍| 485/500 [08:41<00:16,  1.08s/it]textattack: Saving checkpoint under "checkpoints\1720199244498.ta.chkpt" at 2024-07-06 01:07:24 after 485 attacks.
[Succeeded / Failed / Skipped / Total] 322 / 16 / 148 / 486:  97%|██████████████████▍| 486/500 [08:41<00:15,  1.07s/it]

[Succeeded / Failed / Skipped / Total] 325 / 16 / 149 / 490:  98%|██████████████████▌| 490/500 [08:46<00:10,  1.07s/it]textattack: Saving checkpoint under "checkpoints\1720199248999.ta.chkpt" at 2024-07-06 01:07:28 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 325 / 16 / 150 / 491:  98%|██████████████████▋| 491/500 [08:46<00:09,  1.07s/it]

[Succeeded / Failed / Skipped / Total] 328 / 16 / 151 / 495:  99%|██████████████████▊| 495/500 [08:50<00:05,  1.07s/it]textattack: Saving checkpoint under "checkpoints\1720199253434.ta.chkpt" at 2024-07-06 01:07:33 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 330 / 17 / 153 / 500: 100%|███████████████████| 500/500 [08:56<00:00,  1.07s/it]textattack: Saving checkpoint under "checkpoints\1720199259657.ta.chkpt" at 2024-07-06 01:07:39 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 330 / 17 / 153 / 500: 100%|███████████████████| 500/500 [08:56<00:00,  1.07s/it]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 330   |
| Number of failed attacks:     | 17    |
| Number of skipped attacks:    | 153   |
| Original accuracy:            | 69.4% |
| Accuracy under attack:        | 3.4%  |
| Attack success rate:          | 95.1% |
| Average perturbed word %:     | 9.26% |
| Average num. words per input: | 18.4  |
| Avg num queries:              | 32.6  |
+-------------------------------+-------+


In [15]:
runAttack(bert_model_wrapper_svc_mr, n_samples, 'attack_results_new/bertmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/bertmrattacksvcsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 0 / 1 / 5:   1%|▎                           | 5/500 [00:22<37:01,  4.49s/it]textattack: Saving checkpoint under "checkpoints\1720199282190.ta.chkpt" at 2024-07-06 01:08:02 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:   1%|▍                           | 7/500 [00:22<26:33,  3.23s/it]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10:   2%|▌                         | 10/500 [00:29<23:44,  2.91s/it]textattack: Saving checkpoint under "checkpoints\1720199288831.ta.chkpt" at 2024-07-06 01:08:08 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 0 / 4 / 15:   3%|▊                        | 15/500 [00:52<28:32,  3.53s/it]textattack: Saving checkpoint under "checkpoints\1720199312703.ta.chkpt" at 2024-07-06 01:08:32 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 352 / 37 / 96 / 485:  97%|███████████████████▍| 485/500 [14:05<00:26,  1.74s/it]textattack: Saving checkpoint under "checkpoints\1720200104908.ta.chkpt" at 2024-07-06 01:21:44 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 356 / 37 / 97 / 490:  98%|███████████████████▌| 490/500 [14:13<00:17,  1.74s/it]textattack: Saving checkpoint under "checkpoints\1720200113734.ta.chkpt" at 2024-07-06 01:21:53 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 361 / 37 / 97 / 495:  99%|███████████████████▊| 495/500 [14:20<00:08,  1.74s/it]textattack: Saving checkpoint under "checkpoints\1720200120719.ta.chkpt" at 2024-07-06 01:22:00 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 365 / 38 / 97 / 500: 100%|████████████████████| 500/500 [14:34<00:00,  1.75s/it]textattack: Saving checkpoint under "checkpoints\1720200134376.ta.chkpt" at 2024-07-06 01:22:14 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 365 / 38 / 97 / 500: 100%|████████████████████| 500/500 [14:34<00:00,  1.75s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 365    |
| Number of failed attacks:     | 38     |
| Number of skipped attacks:    | 97     |
| Original accuracy:            | 80.6%  |
| Accuracy under attack:        | 7.6%   |
| Attack success rate:          | 90.57% |
| Average perturbed word %:     | 18.74% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 46.12  |
+-------------------------------+--------+


### DistilBERT-based-uncased

In [16]:
# bert-based-uncased
model_path2 = 'distilbert/distilbert-base-uncased'

disbert_model = transformers.AutoModel.from_pretrained(model_path2)
disbert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path2)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
disbertmrtrain = pd.read_csv('dataset_embeddings/disbertmrtrain.txt', header=None)
disbertmrtest = pd.read_csv('dataset_embeddings/disbertmrtest.txt', header=None)
lr_disbert_mr, rf_disbert_mr, knn_disbert_mr, svc_disbert_mr = adaptation(disbertmrtrain, disbertmrtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Log Reg 0.7945590994371482
RF 0.7682926829268293
KNN 0.7298311444652908
SVC 0.7851782363977486


In [18]:
# custom model wrapper for DistilBERT
class DisBERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).last_hidden_state.mean(axis=1)
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [19]:
disbert_model_wrapper_lr_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, lr_disbert_mr)
disbert_model_wrapper_rf_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, rf_disbert_mr)
disbert_model_wrapper_knn_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, knn_disbert_mr)
disbert_model_wrapper_svc_mr = DisBERTModelWrapper(disbert_tokenizer, disbert_model, svc_disbert_mr)

In [20]:
runAttack(disbert_model_wrapper_lr_mr, n_samples, 'attack_results_new/disbertmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacklrsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 3 / 1 / 1 / 5:   1%|▎                           | 5/500 [00:13<22:14,  2.70s/it]textattack: Saving checkpoint under "checkpoints\1720200298463.ta.chkpt" at 2024-07-06 01:24:58 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 1 / 2 / 6:   1%|▎                           | 6/500 [00:13<18:44,  2.28s/it]

[Succeeded / Failed / Skipped / Total] 6 / 1 / 3 / 10:   2%|▌                         | 10/500 [00:18<14:54,  1.82s/it]textattack: Saving checkpoint under "checkpoints\1720200303229.ta.chkpt" at 2024-07-06 01:25:03 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 2 / 4 / 15:   3%|▊                         | 15/500 [00:29<15:43,  1.94s/it]textattack: Saving checkpoint under "checkpoints\1720200314151.ta.chkpt" at 2024-07-06 01:25:14 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 348 / 35 / 102 / 485:  97%|██████████████████▍| 485/500 [08:04<00:14,  1.00it/s]textattack: Saving checkpoint under "checkpoints\1720200769530.ta.chkpt" at 2024-07-06 01:32:49 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 352 / 35 / 103 / 490:  98%|██████████████████▌| 490/500 [08:08<00:09,  1.00it/s]textattack: Saving checkpoint under "checkpoints\1720200773668.ta.chkpt" at 2024-07-06 01:32:53 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 352 / 35 / 104 / 491:  98%|██████████████████▋| 491/500 [08:08<00:08,  1.00it/s]

[Succeeded / Failed / Skipped / Total] 356 / 35 / 104 / 495:  99%|██████████████████▊| 495/500 [08:14<00:04,  1.00it/s]textattack: Saving checkpoint under "checkpoints\1720200779250.ta.chkpt" at 2024-07-06 01:32:59 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 361 / 35 / 104 / 500: 100%|███████████████████| 500/500 [08:22<00:00,  1.00s/it]textattack: Saving checkpoint under "checkpoints\1720200787407.ta.chkpt" at 2024-07-06 01:33:07 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 361 / 35 / 104 / 500: 100%|███████████████████| 500/500 [08:22<00:00,  1.00s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 361    |
| Number of failed attacks:     | 35     |
| Number of skipped attacks:    | 104    |
| Original accuracy:            | 79.2%  |
| Accuracy under attack:        | 7.0%   |
| Attack success rate:          | 91.16% |
| Average perturbed word %:     | 18.74% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 45.8   |
+-------------------------------+--------+


In [21]:
runAttack(disbert_model_wrapper_rf_mr, n_samples, 'attack_results_new/disbertmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackrfsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 2 / 2 / 1 / 5:   1%|▎                           | 5/500 [00:13<22:00,  2.67s/it]textattack: Saving checkpoint under "checkpoints\1720200800836.ta.chkpt" at 2024-07-06 01:33:20 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 2 / 2 / 6:   1%|▎                           | 6/500 [00:13<18:29,  2.25s/it]

[Succeeded / Failed / Skipped / Total] 6 / 2 / 2 / 10:   2%|▌                         | 10/500 [00:20<16:48,  2.06s/it]textattack: Saving checkpoint under "checkpoints\1720200808086.ta.chkpt" at 2024-07-06 01:33:28 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 4 / 3 / 15:   3%|▊                         | 15/500 [00:31<16:58,  2.10s/it]textattack: Saving checkpoint under "checkpoints\1720200819018.ta.chkpt" at 2024-07-06 01:33:39 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 301 / 67 / 117 / 485:  97%|██████████████████▍| 485/500 [08:58<00:16,  1.11s/it]textattack: Saving checkpoint under "checkpoints\1720201325570.ta.chkpt" at 2024-07-06 01:42:05 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 305 / 67 / 118 / 490:  98%|██████████████████▌| 490/500 [09:04<00:11,  1.11s/it]textattack: Saving checkpoint under "checkpoints\1720201331805.ta.chkpt" at 2024-07-06 01:42:11 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 305 / 67 / 119 / 491:  98%|██████████████████▋| 491/500 [09:04<00:09,  1.11s/it]

[Succeeded / Failed / Skipped / Total] 308 / 67 / 120 / 495:  99%|██████████████████▊| 495/500 [09:07<00:05,  1.11s/it]textattack: Saving checkpoint under "checkpoints\1720201335049.ta.chkpt" at 2024-07-06 01:42:15 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 313 / 67 / 120 / 500: 100%|███████████████████| 500/500 [09:17<00:00,  1.11s/it]textattack: Saving checkpoint under "checkpoints\1720201344628.ta.chkpt" at 2024-07-06 01:42:24 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 313 / 67 / 120 / 500: 100%|███████████████████| 500/500 [09:17<00:00,  1.11s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 313    |
| Number of failed attacks:     | 67     |
| Number of skipped attacks:    | 120    |
| Original accuracy:            | 76.0%  |
| Accuracy under attack:        | 13.4%  |
| Attack success rate:          | 82.37% |
| Average perturbed word %:     | 17.99% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 50.78  |
+-------------------------------+--------+


In [22]:
runAttack(disbert_model_wrapper_knn_mr, n_samples, 'attack_results_new/disbertmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattackknnsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 4 / 1 / 0 / 5:   1%|▎                           | 5/500 [00:12<20:45,  2.52s/it]textattack: Saving checkpoint under "checkpoints\1720201357270.ta.chkpt" at 2024-07-06 01:42:37 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 1 / 2 / 7:   1%|▍                           | 7/500 [00:12<14:55,  1.82s/it]

[Succeeded / Failed / Skipped / Total] 5 / 3 / 2 / 10:   2%|▌                         | 10/500 [00:20<16:41,  2.04s/it]textattack: Saving checkpoint under "checkpoints\1720201365131.ta.chkpt" at 2024-07-06 01:42:45 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 5 / 3 / 15:   3%|▊                         | 15/500 [00:25<13:56,  1.73s/it]textattack: Saving checkpoint under "checkpoints\1720201370569.ta.chkpt" at 2024-07-06 01:42:50 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 289 / 68 / 128 / 485:  97%|██████████████████▍| 485/500 [08:12<00:15,  1.02s/it]textattack: Saving checkpoint under "checkpoints\1720201837574.ta.chkpt" at 2024-07-06 01:50:37 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 293 / 68 / 129 / 490:  98%|██████████████████▌| 490/500 [08:16<00:10,  1.01s/it]textattack: Saving checkpoint under "checkpoints\1720201841684.ta.chkpt" at 2024-07-06 01:50:41 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 295 / 69 / 131 / 495:  99%|██████████████████▊| 495/500 [08:24<00:05,  1.02s/it]textattack: Saving checkpoint under "checkpoints\1720201849576.ta.chkpt" at 2024-07-06 01:50:49 after 495 attacks.
[Succeeded / Failed / Skipped / Total] 295 / 69 / 132 / 496:  99%|██████████████████▊| 496/500 [08:24<00:04,  1.02s/it]

[Succeeded / Failed / Skipped / Total] 296 / 72 / 132 / 500: 100%|███████████████████| 500/500 [08:36<00:00,  1.03s/it]textattack: Saving checkpoint under "checkpoints\1720201861061.ta.chkpt" at 2024-07-06 01:51:01 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 296 / 72 / 132 / 500: 100%|███████████████████| 500/500 [08:36<00:00,  1.03s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 296    |
| Number of failed attacks:     | 72     |
| Number of skipped attacks:    | 132    |
| Original accuracy:            | 73.6%  |
| Accuracy under attack:        | 14.4%  |
| Attack success rate:          | 80.43% |
| Average perturbed word %:     | 11.09% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 47.58  |
+-------------------------------+--------+


In [23]:
runAttack(disbert_model_wrapper_svc_mr, n_samples, 'attack_results_new/disbertmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.distilbert.modeling_distilbert.DistilBertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/disbertmrattacksvcsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 3 / 1 / 1 / 5:   1%|▎                           | 5/500 [00:12<20:12,  2.45s/it]textattack: Saving checkpoint under "checkpoints\1720201873492.ta.chkpt" at 2024-07-06 01:51:13 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 1 / 3 / 7:   1%|▍                           | 7/500 [00:12<14:35,  1.78s/it]

[Succeeded / Failed / Skipped / Total] 6 / 1 / 3 / 10:   2%|▌                         | 10/500 [00:17<14:08,  1.73s/it]textattack: Saving checkpoint under "checkpoints\1720201878568.ta.chkpt" at 2024-07-06 01:51:18 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 2 / 4 / 15:   3%|▊                         | 15/500 [00:28<15:18,  1.89s/it]textattack: Saving checkpoint under "checkpoints\1720201889648.ta.chkpt" at 2024-07-06 01:51:29 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 343 / 38 / 104 / 485:  97%|██████████████████▍| 485/500 [08:06<00:15,  1.00s/it]textattack: Saving checkpoint under "checkpoints\1720202348033.ta.chkpt" at 2024-07-06 01:59:08 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 345 / 39 / 106 / 490:  98%|██████████████████▌| 490/500 [08:10<00:10,  1.00s/it]textattack: Saving checkpoint under "checkpoints\1720202351908.ta.chkpt" at 2024-07-06 01:59:11 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 345 / 39 / 107 / 491:  98%|██████████████████▋| 491/500 [08:10<00:08,  1.00it/s]

[Succeeded / Failed / Skipped / Total] 349 / 39 / 107 / 495:  99%|██████████████████▊| 495/500 [08:16<00:05,  1.00s/it]textattack: Saving checkpoint under "checkpoints\1720202357455.ta.chkpt" at 2024-07-06 01:59:17 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 354 / 39 / 107 / 500: 100%|███████████████████| 500/500 [08:24<00:00,  1.01s/it]textattack: Saving checkpoint under "checkpoints\1720202365972.ta.chkpt" at 2024-07-06 01:59:25 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 354 / 39 / 107 / 500: 100%|███████████████████| 500/500 [08:24<00:00,  1.01s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 354    |
| Number of failed attacks:     | 39     |
| Number of skipped attacks:    | 107    |
| Original accuracy:            | 78.6%  |
| Accuracy under attack:        | 7.8%   |
| Attack success rate:          | 90.08% |
| Average perturbed word %:     | 18.0%  |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 45.04  |
+-------------------------------+--------+


### CLIP

In [24]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
clipmrtrain = pd.read_csv('dataset_embeddings/clipmrtrain.txt', header=None)
clipmrtest = pd.read_csv('dataset_embeddings/clipmrtest.txt', header=None)
lr_clip_mr, rf_clip_mr, knn_clip_mr, svc_clip_mr = adaptation(clipmrtrain, clipmrtest)

Log Reg 0.7495309568480301
RF 0.7054409005628518
KNN 0.6885553470919324
SVC 0.7551594746716698


In [26]:
# custom model wrapper for CLIP
class CLIPModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [27]:
clip_model_wrapper_lr_mr = CLIPModelWrapper(clip_processor, text_encoder, lr_clip_mr)
clip_model_wrapper_rf_mr = CLIPModelWrapper(clip_processor, text_encoder, rf_clip_mr)
clip_model_wrapper_knn_mr = CLIPModelWrapper(clip_processor, text_encoder, knn_clip_mr)
clip_model_wrapper_svc_mr = CLIPModelWrapper(clip_processor, text_encoder, svc_clip_mr)

In [28]:
runAttack(clip_model_wrapper_lr_mr, n_samples, 'attack_results_new/clipmrattacklr'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacklrsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   1%|▎                           | 5/500 [00:13<22:42,  2.75s/it]textattack: Saving checkpoint under "checkpoints\1720202779843.ta.chkpt" at 2024-07-06 02:06:19 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 1 / 6:   1%|▎                           | 6/500 [00:13<18:58,  2.30s/it]

[Succeeded / Failed / Skipped / Total] 9 / 0 / 1 / 10:   2%|▌                         | 10/500 [00:17<14:14,  1.74s/it]textattack: Saving checkpoint under "checkpoints\1720202783515.ta.chkpt" at 2024-07-06 02:06:23 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 1 / 1 / 15:   3%|▊                        | 15/500 [00:29<15:55,  1.97s/it]textattack: Saving checkpoint under "checkpoints\1720202795641.ta.chkpt" at 2024-07-06 02:06:35 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 341 / 17 / 127 / 485:  97%|██████████████████▍| 485/500 [06:59<00:12,  1.16it/s]textattack: Saving checkpoint under "checkpoints\1720203185672.ta.chkpt" at 2024-07-06 02:13:05 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 344 / 17 / 129 / 490:  98%|██████████████████▌| 490/500 [07:02<00:08,  1.16it/s]textattack: Saving checkpoint under "checkpoints\1720203188907.ta.chkpt" at 2024-07-06 02:13:08 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 344 / 17 / 130 / 491:  98%|██████████████████▋| 491/500 [07:02<00:07,  1.16it/s]

[Succeeded / Failed / Skipped / Total] 347 / 17 / 131 / 495:  99%|██████████████████▊| 495/500 [07:04<00:04,  1.16it/s]textattack: Saving checkpoint under "checkpoints\1720203190969.ta.chkpt" at 2024-07-06 02:13:10 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 352 / 17 / 131 / 500: 100%|███████████████████| 500/500 [07:10<00:00,  1.16it/s]textattack: Saving checkpoint under "checkpoints\1720203196830.ta.chkpt" at 2024-07-06 02:13:16 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 352 / 17 / 131 / 500: 100%|███████████████████| 500/500 [07:10<00:00,  1.16it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 352    |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 131    |
| Original accuracy:            | 73.8%  |
| Accuracy under attack:        | 3.4%   |
| Attack success rate:          | 95.39% |
| Average perturbed word %:     | 17.04% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 40.17  |
+-------------------------------+--------+


In [29]:
runAttack(clip_model_wrapper_rf_mr, n_samples, 'attack_results_new/clipmrattackrf'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackrfsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 2 / 3 / 0 / 5:   1%|▎                           | 5/500 [00:15<24:56,  3.02s/it]textattack: Saving checkpoint under "checkpoints\1720203212051.ta.chkpt" at 2024-07-06 02:13:32 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 3 / 2 / 7:   1%|▍                           | 7/500 [00:15<17:54,  2.18s/it]

[Succeeded / Failed / Skipped / Total] 5 / 3 / 2 / 10:   2%|▌                         | 10/500 [00:19<15:51,  1.94s/it]textattack: Saving checkpoint under "checkpoints\1720203216333.ta.chkpt" at 2024-07-06 02:13:36 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 5 / 3 / 15:   3%|▊                         | 15/500 [00:29<15:52,  1.96s/it]textattack: Saving checkpoint under "checkpoints\1720203226383.ta.chkpt" at 2024-07-06 02:13:46 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 293 / 48 / 144 / 485:  97%|██████████████████▍| 485/500 [08:13<00:15,  1.02s/it]textattack: Saving checkpoint under "checkpoints\1720203690583.ta.chkpt" at 2024-07-06 02:21:30 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 297 / 48 / 145 / 490:  98%|██████████████████▌| 490/500 [08:17<00:10,  1.01s/it]textattack: Saving checkpoint under "checkpoints\1720203693974.ta.chkpt" at 2024-07-06 02:21:33 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 297 / 48 / 146 / 491:  98%|██████████████████▋| 491/500 [08:17<00:09,  1.01s/it]

[Succeeded / Failed / Skipped / Total] 299 / 48 / 148 / 495:  99%|██████████████████▊| 495/500 [08:21<00:05,  1.01s/it]textattack: Saving checkpoint under "checkpoints\1720203698756.ta.chkpt" at 2024-07-06 02:21:38 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 303 / 48 / 149 / 500: 100%|███████████████████| 500/500 [08:25<00:00,  1.01s/it]textattack: Saving checkpoint under "checkpoints\1720203702631.ta.chkpt" at 2024-07-06 02:21:42 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 303 / 48 / 149 / 500: 100%|███████████████████| 500/500 [08:25<00:00,  1.01s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 303    |
| Number of failed attacks:     | 48     |
| Number of skipped attacks:    | 149    |
| Original accuracy:            | 70.2%  |
| Accuracy under attack:        | 9.6%   |
| Attack success rate:          | 86.32% |
| Average perturbed word %:     | 14.84% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 45.04  |
+-------------------------------+--------+


In [30]:
runAttack(clip_model_wrapper_knn_mr, n_samples, 'attack_results_new/clipmrattackknn'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattackknnsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 3 / 2 / 0 / 5:   1%|▎                           | 5/500 [00:14<23:26,  2.84s/it]textattack: Saving checkpoint under "checkpoints\1720203716927.ta.chkpt" at 2024-07-06 02:21:56 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 3 / 2 / 10:   2%|▌                         | 10/500 [00:19<15:47,  1.93s/it]textattack: Saving checkpoint under "checkpoints\1720203722053.ta.chkpt" at 2024-07-06 02:22:02 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 4 / 3 / 15:   3%|▊                         | 15/500 [00:25<13:58,  1.73s/it]textattack: Saving checkpoint under "checkpoints\1720203728662.ta.chkpt" at 2024-07-06 02:22:08 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 279 / 54 / 152 / 485:  97%|██████████████████▍| 485/500 [07:40<00:14,  1.05it/s]textattack: Saving checkpoint under "checkpoints\1720204163462.ta.chkpt" at 2024-07-06 02:29:23 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 282 / 55 / 153 / 490:  98%|██████████████████▌| 490/500 [07:47<00:09,  1.05it/s]textattack: Saving checkpoint under "checkpoints\1720204169775.ta.chkpt" at 2024-07-06 02:29:29 after 490 attacks.
[Succeeded / Failed / Skipped / Total] 282 / 55 / 154 / 491:  98%|██████████████████▋| 491/500 [07:47<00:08,  1.05it/s]

[Succeeded / Failed / Skipped / Total] 284 / 55 / 156 / 495:  99%|██████████████████▊| 495/500 [07:48<00:04,  1.06it/s]textattack: Saving checkpoint under "checkpoints\1720204171697.ta.chkpt" at 2024-07-06 02:29:31 after 495 attacks.
[Succeeded / Failed / Skipped / Total] 284 / 55 / 158 / 497:  99%|██████████████████▉| 497/500 [07:49<00:02,  1.06it/s]

[Succeeded / Failed / Skipped / Total] 287 / 55 / 158 / 500: 100%|███████████████████| 500/500 [07:51<00:00,  1.06it/s]textattack: Saving checkpoint under "checkpoints\1720204173854.ta.chkpt" at 2024-07-06 02:29:33 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 287 / 55 / 158 / 500: 100%|███████████████████| 500/500 [07:51<00:00,  1.06it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 287    |
| Number of failed attacks:     | 55     |
| Number of skipped attacks:    | 158    |
| Original accuracy:            | 68.4%  |
| Accuracy under attack:        | 11.0%  |
| Attack success rate:          | 83.92% |
| Average perturbed word %:     | 11.05% |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 42.85  |
+-------------------------------+--------+


In [31]:
runAttack(clip_model_wrapper_svc_mr, n_samples, 'attack_results_new/clipmrattacksvc'+suffix, target_ds_mr)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path attack_results_new/clipmrattacksvcsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

[Succeeded / Failed / Skipped / Total] 5 / 0 / 0 / 5:   1%|▎                           | 5/500 [00:11<18:45,  2.27s/it]textattack: Saving checkpoint under "checkpoints\1720204185366.ta.chkpt" at 2024-07-06 02:29:45 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 2 / 7:   1%|▍                           | 7/500 [00:11<13:29,  1.64s/it]

[Succeeded / Failed / Skipped / Total] 8 / 0 / 2 / 10:   2%|▌                         | 10/500 [00:14<11:45,  1.44s/it]textattack: Saving checkpoint under "checkpoints\1720204188397.ta.chkpt" at 2024-07-06 02:29:48 after 10 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 0 / 3 / 15:   3%|▊                        | 15/500 [00:23<12:39,  1.57s/it]textattack: Saving checkpoint under "checkpoints\1720204197475.ta.chkpt" at 2024-07-06 02:29:57 after 15 attacks.


...

[Succeeded / Failed / Skipped / Total] 343 / 17 / 125 / 485:  97%|██████████████████▍| 485/500 [07:10<00:13,  1.13it/s]textattack: Saving checkpoint under "checkpoints\1720204604437.ta.chkpt" at 2024-07-06 02:36:44 after 485 attacks.


[Succeeded / Failed / Skipped / Total] 345 / 17 / 128 / 490:  98%|██████████████████▌| 490/500 [07:13<00:08,  1.13it/s]textattack: Saving checkpoint under "checkpoints\1720204607876.ta.chkpt" at 2024-07-06 02:36:47 after 490 attacks.


[Succeeded / Failed / Skipped / Total] 349 / 17 / 129 / 495:  99%|██████████████████▊| 495/500 [07:16<00:04,  1.13it/s]textattack: Saving checkpoint under "checkpoints\1720204610938.ta.chkpt" at 2024-07-06 02:36:50 after 495 attacks.


[Succeeded / Failed / Skipped / Total] 354 / 17 / 129 / 500: 100%|███████████████████| 500/500 [07:23<00:00,  1.13it/s]textattack: Saving checkpoint under "checkpoints\1720204617736.ta.chkpt" at 2024-07-06 02:36:57 after 500 attacks.
[Succeeded / Failed / Skipped / Total] 354 / 17 / 129 / 500: 100%|███████████████████| 500/500 [07:23<00:00,  1.13it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 354    |
| Number of failed attacks:     | 17     |
| Number of skipped attacks:    | 129    |
| Original accuracy:            | 74.2%  |
| Accuracy under attack:        | 3.4%   |
| Attack success rate:          | 95.42% |
| Average perturbed word %:     | 16.6%  |
| Average num. words per input: | 18.4   |
| Avg num queries:              | 39.49  |
+-------------------------------+--------+
